# Run Notebooks on a schedule

A Cat Breed Classifier that

1. Downloads a cat image from catapi.com
2. Uses a Tensorflow model to identify the breed of the cat

In [ ]:
import os
import requests
from PIL import Image
from io import BytesIO
# Suppress tf error messages related to GPU access
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_api_key(file_path):
    """Load the API key from a file."""
    try:
        with open(file_path, 'r') as file:
            return file.readline().strip()
    except FileNotFoundError:
        raise Exception("API key file not found.")
    except Exception as e:
        raise Exception(f"An error occurred while reading the API key: {str(e)}")

api_key_path = os.path.expanduser('~/shared/ai-research/catapi/api_key.txt')
api_key = load_api_key(api_key_path)

In [ ]:
model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=True)

In [ ]:
def prepare_image(image_url):
    headers = {'x-api-key': api_key}  # Set the API key in headers
    response = requests.get(image_url, headers=headers)
    image = Image.open(BytesIO(response.content))
    image = image.resize((224, 224))
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = tf.keras.applications.mobilenet_v2.preprocess_input(image_array)
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

def fetch_image_url():
    api_url = 'https://api.thecatapi.com/v1/images/search?size=med&mime_types=jpg&format=json&has_breeds=true&order=RANDOM&page=0&limit=1'
    headers = {'x-api-key': api_key}
    response = requests.get(api_url, headers=headers)
    image_data = response.json()
    if image_data:
        return image_data[0]['url']
    return None

def classify_image(image_url):
    image_array = prepare_image(image_url)
    predictions = model.predict(image_array)
    decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=1)  # Get top 1 prediction
    breed, probability = decoded_predictions[0][0][1], decoded_predictions[0][0][2]
    return breed, probability

def display_image_with_prediction():
    image_url = fetch_image_url()
    if image_url:
        breed, probability = classify_image(image_url)
        # Fetch the image again to display
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content))

        # Plotting
        plt.imshow(image)
        plt.axis('off')  # Turn off axis
        plt.title(f"{breed} ({probability*100:.2f}%)")
        plt.show()
    else:
        print("No image found.")

In [ ]:
display_image_with_prediction()